# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである。体系的にまとまってはいない。

## 1. 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける。

関数内で配列を変更する場合には注意が必要である。以下に入力された配列を同じサイズの要素1の配列で置き換える、ということを目的として書かれた2つの関数がある。違いは`v`の後に`[:]`としているかどうかである。

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる。

In [2]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : 

[0.3090368990432588 0.28505372035635523; 0.36039892643656835 0.678223689591329]
wrong : [0.3090368990432588 0.28505372035635523; 0.36039892643656835 0.678223689591329]
right : [1.0 1.0; 1.0 1.0]


## 2. 配列の1次元化
配列を一次元化(flatten)する方法。まずは3次元配列を作成する。

In [3]:
B = rand(2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.00534986  0.4856
 0.438555    0.744457

[:, :, 2] =
 0.810031  0.162148
 0.753313  0.0655913

用意されている`flatten`を素直に用いると次のようになる。

In [4]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Array{Float64,1}:
 0.005349859924772105
 0.43855482864877393
 0.48559963845290754
 0.7444570141514844
 0.8100312582206968
 0.7533126121452021
 0.1621477669172322
 0.06559133429633146

ただし、単に`B[:]`とするだけでもよい。

In [5]:
B[:]

8-element Array{Float64,1}:
 0.005349859924772105
 0.43855482864877393
 0.48559963845290754
 0.7444570141514844
 0.8100312582206968
 0.7533126121452021
 0.1621477669172322
 0.06559133429633146

## 3. 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある。これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある。以下では行ごとの和を1にする。

In [6]:
W = rand(3,3)

3×3 Array{Float64,2}:
 0.149263  0.842437   0.690403
 0.497057  0.41185    0.638995
 0.757788  0.0330631  4.52843e-5

In [7]:
Wnormed = W ./ sum(W, dims=1)

3×3 Array{Float64,2}:
 0.106305  0.654396   0.519318
 0.354002  0.319921   0.480648
 0.539693  0.0256831  3.40626e-5

In [8]:
println(sum(Wnormed, dims=1))

[1.0 1.0 0.9999999999999999]


## 4. 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる。まず、2つの行列A, Bを用意する。

In [9]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [10]:
B = [4 5 6; 7 8 9]

2×3 Array{Int64,2}:
 4  5  6
 7  8  9

### 4.1 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と、`[ ]`を使うやり方がある。

In [11]:
H1 = hcat(A,B)

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

In [12]:
H2 = [A B]

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

なお、MATLABのように次のようにすると正しく結合はされない。

In [13]:
H3 = [A, B]

2-element Array{Array{Int64,2},1}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 4.2 垂直結合 (Vertical concatenation)

In [14]:
V1 = vcat(A, B')

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [15]:
V2 = [A; B']

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [16]:
[V2 [A;B']]

5×4 Array{Int64,2}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 5. 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合。やや面倒だが、`reshape`を使うか、`[CartesianIndex()]`を用いる。

In [17]:
v = rand(3)

3-element Array{Float64,1}:
 0.9700309221959584
 0.9191627169218128
 0.8465877425923181

In [18]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Array{Float64,2}:
 0.970031  0.919163  0.846588

## 6. Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると、1次元配列のリストを2次元配列に変換できた。Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる。

In [19]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Array{Array{Float64,1},1}
Size : (5,)


In [20]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64,Array{Float64,2}}
Size : (5, 3)


以下は多次元配列の場合。`cat(...)`で配列を結合し、`permitedims`で転置する。

In [21]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Array{Array{Float64,3},1}
Size : (6,)


In [22]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64,4}
Size : (6, 3, 4, 5)


## 7. 二項分布 (bernoulli distribution)のサンプリング

In [23]:
p = 0.7
N = 100

100

In [24]:
using BenchmarkTools

In [25]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.91 KiB
  allocs estimate:  7
  --------------
  minimum time:     686.099 ns (0.00% GC)
  median time:      715.894 ns (0.00% GC)
  mean time:        760.233 ns (3.12% GC)
  maximum time:     5.828 μs (84.17% GC)
  --------------
  samples:          10000
  evals/sample:     151

In [26]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.61


In [27]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.55 KiB
  allocs estimate:  6
  --------------
  minimum time:     868.983 ns (0.00% GC)
  median time:      922.414 ns (0.00% GC)
  mean time:        976.752 ns (2.83% GC)
  maximum time:     16.453 μs (87.74% GC)
  --------------
  samples:          10000
  evals/sample:     58

In [28]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.73
